https://www.kaggle.com/husterwgm/keras-neural-net


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

In [2]:
train=pd.read_csv('D:/bnp-paribas-cardif-claims-management/train.csv')
test=pd.read_csv('D:/bnp-paribas-cardif-claims-management/test.csv')
submission=pd.read_csv('D:/bnp-paribas-cardif-claims-management/sample_submission.csv')


In [3]:
target=train['target']
df_train=train.drop(['target'],axis=1)

id_test=test['ID'].values

In [7]:
drop_features=['v8','v23','v25','v36','v37','v46','v51','v53','v54','v63','v73','v75','v79','v81','v82','v89','v92','v95','v105','v107','v108','v109','v110','v116','v117','v118','v119','v123','v124','v128']

In [9]:
from sklearn.preprocessing import LabelEncoder
for col in df_train.select_dtypes(include='object').columns:
    df_train[col]=pd.factorize(df_train[col])[0]
    test[col]=pd.factorize(test[col])[0]

In [11]:
sparse_col=[]
for col in df_train.columns:
    cls=df_train[col].values
    if sum(cls)<10:
        sparse_col.append(col)

In [14]:
df_train.drop(sparse_col,axis=1,inplace=True)
test.drop(sparse_col,axis=1,inplace=True)

In [15]:
df_train=df_train.fillna(-1)
test=test.fillna(-1)

In [20]:
feature_name=df_train.columns.values.tolist()
X_train=df_train.values
X_test=test.values

In [22]:
print(X_train.shape)
print(X_test.shape)

(114321, 130)
(114393, 130)


In [23]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [39]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val=train_test_split(X_train,target,test_size=0.2,random_state=0)

In [64]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

model=Sequential()
model.add(Dense(50,input_shape=(X_tr.shape[1],),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(40,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [65]:
model.fit(X_tr,y_tr,epochs=20,batch_size=128,validation_data=[X_val,y_val],verbose=1)

Train on 91456 samples, validate on 22865 samples
Epoch 1/20
91456/91456 [==============================] - 6s 65us/step - loss: 0.5242 - acc: 0.7569 - val_loss: 0.4942 - val_acc: 0.7654
Epoch 2/20
91456/91456 [==============================] - 4s 49us/step - loss: 0.5022 - acc: 0.7628 - val_loss: 0.4955 - val_acc: 0.7679
Epoch 3/20
91456/91456 [==============================] - 4s 49us/step - loss: 0.4977 - acc: 0.7648 - val_loss: 0.4859 - val_acc: 0.7738
Epoch 4/20
91456/91456 [==============================] - 4s 49us/step - loss: 0.4937 - acc: 0.7675 - val_loss: 0.4894 - val_acc: 0.7742
Epoch 5/20
91456/91456 [==============================] - 4s 48us/step - loss: 0.4915 - acc: 0.7695 - val_loss: 0.4855 - val_acc: 0.7760
Epoch 6/20
91456/91456 [==============================] - 4s 46us/step - loss: 0.4903 - acc: 0.7701 - val_loss: 0.4873 - val_acc: 0.7769
Epoch 7/20
91456/91456 [==============================] - 4s 46us/step - loss: 0.4884 - acc: 0.7719 - val_loss: 0.4870 - val_acc

In [66]:
from sklearn.metrics import confusion_matrix,log_loss

y_val_pred=model.predict(X_val)
#y_val_pred=np.where(y_val_pred>=0.5,1,0)
logloss=log_loss(y_val,y_val_pred)
print(logloss)

0.4877626049272363


In [61]:
y_pred=model.predict_proba(X_test)[:,0]

In [62]:
sub=pd.DataFrame({'ID':id_test,'PredictedProb':y_pred})
sub.to_csv('neural_net.csv',index=False)

In [63]:
sub.head()

,ID,PredictedProb
0,0,0.652974
1,1,0.814949
2,2,0.587090
3,7,0.670690
4,10,0.810675
